In [1]:
# import warnings
# warnings.filterwarnings("ignore")

import csv
import numpy as np
import pandas as pd
import scipy

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [16]:
im_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [133]:
a_dir = "/Users/hn/Documents/01_research_data/NASA/"
param_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/" 
evaluation_set = pd.read_csv(param_dir + "evaluation_set.csv", low_memory=False)

In [5]:
evaluation_set.head(2)

,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


In [20]:
all_files = [x for x in os.listdir(im_dir) if x.endswith(".png")]

In [35]:
TOA_files = [x for x in all_files if ("TOA" in x)]
corrected_files = [x for x in all_files if ("corrected" in x)]

NDVIs = [x for x in all_files if not("corrected" in x)]
NDVIs = [x for x in NDVIs if not("TOA" in x)]

In [116]:
df = pd.DataFrame(index=range(len(NDVIs)), 
                   columns=["ID", 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', "latitude", "longitude"])

for counter, file in enumerate(NDVIs):
    L = file.split("_")
    L.pop(0)
    field_ID = "_".join(L)[:-4]
    # print (file, counter, field_ID, sep = ", ")
    
    # pick corrected name
    TOA_file_name = [x for x in TOA_files if field_ID in x]
    correct_file_name = [x for x in corrected_files if field_ID in x]
    
    df.loc[counter, "ID"] = field_ID
    df.loc[counter, "NDVI_TS_Name"] = file
    df.loc[counter, "corrected_RGB"] = correct_file_name[0]
    df.loc[counter, "TOA_RGB"] = TOA_file_name[0]
    
    df.loc[counter, "longitude"] = TOA_file_name[0].split("_")[5]
    df.loc[counter, "latitude"] = TOA_file_name[0].split("_")[4]
    


In [122]:
df.head(3)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude
0,46092_WSDA_SF_2018,FranklinYakima2018_46092_WSDA_SF_2018.png,46092_WSDA_SF_2018_46.38682332_-120.21093938_c...,46092_WSDA_SF_2018_46.38682332_-120.21093938_T...,46.38682332,-120.21093938
1,106176_WSDA_SF_2017,Grant2017_106176_WSDA_SF_2017.png,106176_WSDA_SF_2017_47.26307371_-119.80344481_...,106176_WSDA_SF_2017_47.26307371_-119.80344481_...,47.26307371,-119.80344481
2,100574_WSDA_SF_2017,Grant2017_100574_WSDA_SF_2017.png,100574_WSDA_SF_2017_47.17965268_-119.84781448_...,100574_WSDA_SF_2017_47.17965268_-119.84781448_...,47.17965268,-119.84781448


In [123]:
result = pd.merge(df, evaluation_set, on="ID")
result.head(3)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,46092_WSDA_SF_2018,FranklinYakima2018_46092_WSDA_SF_2018.png,46092_WSDA_SF_2018_46.38682332_-120.21093938_c...,46092_WSDA_SF_2018_46.38682332_-120.21093938_T...,46.38682332,-120.21093938,alfalfa hay,sprinkler,wsda,8,8.186591,2018/10/15 00:00:00,Yakima
1,106176_WSDA_SF_2017,Grant2017_106176_WSDA_SF_2017.png,106176_WSDA_SF_2017_47.26307371_-119.80344481_...,106176_WSDA_SF_2017_47.26307371_-119.80344481_...,47.26307371,-119.80344481,apple,drip/micro-sprinkler,wsda,2,1.759810,2017/05/03,Grant
2,100574_WSDA_SF_2017,Grant2017_100574_WSDA_SF_2017.png,100574_WSDA_SF_2017_47.17965268_-119.84781448_...,100574_WSDA_SF_2017_47.17965268_-119.84781448_...,47.17965268,-119.84781448,"corn, field",rill,wsda,50,49.692536,2017/08/09,Grant


In [128]:
result.sort_values(by=['CropTyp', 'ID'], inplace=True)
result.reset_index(drop=True, inplace=True)
result.head(3)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
194,100010_WSDA_SF_2017,Grant2017_100010_WSDA_SF_2017.png,100010_WSDA_SF_2017_46.82747438_-119.91621244_...,100010_WSDA_SF_2017_46.82747438_-119.91621244_...,46.82747438,-119.91621244,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
5115,100204_WSDA_SF_2017,Grant2017_100204_WSDA_SF_2017.png,100204_WSDA_SF_2017_47.18015347_-119.74905171_...,100204_WSDA_SF_2017_47.18015347_-119.74905171_...,47.18015347,-119.74905171,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant
5944,100214_WSDA_SF_2017,Grant2017_100214_WSDA_SF_2017.png,100214_WSDA_SF_2017_47.15794603_-119.63922834_...,100214_WSDA_SF_2017_47.15794603_-119.63922834_...,47.15794603,-119.63922834,alfalfa hay,center pivot,wsda,13,13.105399,2017/08/28,Grant


In [132]:
eshwar_limited = result[["ID", 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', "latitude", "longitude"]].copy()
eshwar_limited.reset_index(drop=True, inplace=True)
eshwar_limited.head(3)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude
0,100010_WSDA_SF_2017,Grant2017_100010_WSDA_SF_2017.png,100010_WSDA_SF_2017_46.82747438_-119.91621244_...,100010_WSDA_SF_2017_46.82747438_-119.91621244_...,46.82747438,-119.91621244
1,100204_WSDA_SF_2017,Grant2017_100204_WSDA_SF_2017.png,100204_WSDA_SF_2017_47.18015347_-119.74905171_...,100204_WSDA_SF_2017_47.18015347_-119.74905171_...,47.18015347,-119.74905171
2,100214_WSDA_SF_2017,Grant2017_100214_WSDA_SF_2017.png,100214_WSDA_SF_2017_47.15794603_-119.63922834_...,100214_WSDA_SF_2017_47.15794603_-119.63922834_...,47.15794603,-119.63922834


In [142]:
out_name = param_dir + "Eshwar_Extensive.csv"
result.index += 1
result.to_csv(out_name, index = True)

out_name = param_dir + "Eshwar_limited.csv"
eshwar_limited.index += 1
eshwar_limited.to_csv(out_name, index = True)